# Agentic RAG Flow

---

## Introduction

This notebook demonstrates an **Advanced Retrieval-Augmented Generation (RAG)** workflow using [LangGraph](https://www.langchain.com/langgraph). This prototype is designed to:

- Dynamically route questions to either a **vectorstore retriever** or **web search** based on relevance.
- Grade retrieved documents and model generations for **usefulness** and **hallucination detection**.
- Adaptively iterate using feedback and conditional routing to improve responses.

This workflow is inspired by:
- [**Self-RAG**](https://arxiv.org/abs/2310.11511) (Akari Asai et al., 2023)
- [**Adaptive RAG**](https://arxiv.org/abs/2403.14403) (Jeong et al., 2024)
- [LangGraph Udemy course](https://www.udemy.com/course/langgraph).

### System Flow Diagram

Below is the graph representation of the LangGraph workflow:

![LangGraph Workflow](graph.png)

In [1]:
# Environment Setup and Imports

from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from typing import List, TypedDict, Dict, Any, Literal
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnableSequence
from langchain_tavily import TavilySearch
from langgraph.graph import END, StateGraph

load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [ ]:
# Load and Split Documents

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)

In [2]:
# Set Up Vectorstore Retriever

# Uncomment to regenerate vectorstore
# vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=OpenAIEmbeddings(),
#     persist_directory="./.chroma",
# )

retriever = Chroma(
    collection_name="rag-chroma",
    persist_directory="./.chroma",
    embedding_function=OpenAIEmbeddings(),
).as_retriever()

In [4]:
# Graph State Definition, Retrieval Node, and Document Relevance Grader

class GraphState(TypedDict, total=False):
    question: str
    generation: str
    web_search: bool
    documents: List[Document]

def retrieve(state: GraphState) -> Dict[str, Any]:
    print("---RETRIEVE---")
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

class GradeDocuments(BaseModel):
    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

llm = ChatOpenAI(temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

grade_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a grader assessing relevance of a retrieved document to a user question."),
    ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
])
retrieval_grader = grade_prompt | structured_llm_grader

def grade_documents(state: GraphState) -> Dict[str, Any]:
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    filtered_docs = []
    web_search = False
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        if score.binary_score.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = True
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

/home/john/.local/share/virtualenvs/ice_breaker-p8pkssDf/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1685: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [5]:
## ✂Truncate Documents Function, Generation and Web Search Nodes, Answer & Hallucination Graders

def truncate_documents(documents, max_chars=12000):
    total = 0
    result = []
    for d in documents:
        if total + len(d.page_content) > max_chars:
            break
        result.append(d)
        total += len(d.page_content)
    return result

prompt = hub.pull("rlm/rag-prompt")
generation_chain = prompt | llm | StrOutputParser()

def generate(state: GraphState) -> Dict[str, Any]:
    print("---GENERATE---")
    question = state["question"]
    documents = state.get("documents", [])
    trimmed_docs = truncate_documents(documents)
    generation = generation_chain.invoke({"context": trimmed_docs, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

web_search_tool = TavilySearch(max_results=3)

def web_search(state: GraphState) -> Dict[str, Any]:
    print("---WEB SEARCH---")
    question = state["question"]
    tavily_results = web_search_tool.invoke({"query": question})
    if isinstance(tavily_results, list) and isinstance(tavily_results[0], dict):
        joined = "\n".join([r.get("content", "") for r in tavily_results])
    elif isinstance(tavily_results, list):
        joined = "\n".join(tavily_results)
    else:
        joined = str(tavily_results)
    return {"documents": [Document(page_content=joined)], "question": question}

class GradeAnswer(BaseModel):
    binary_score: bool = Field(description="Answer addresses the question, 'yes' or 'no'")

answer_grader = ChatPromptTemplate.from_messages([
    ("system", "You are a grader assessing whether an answer addresses / resolves a question"),
    ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
]) | llm.with_structured_output(GradeAnswer)

class GradeHallucinations(BaseModel):
    binary_score: bool = Field(description="Answer is grounded in the facts, 'yes' or 'no'")

hallucination_grader = ChatPromptTemplate.from_messages([
    ("system", "You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts."),
    ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
]) | llm.with_structured_output(GradeHallucinations)

/home/john/.local/share/virtualenvs/ice_breaker-p8pkssDf/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/home/john/.local/share/virtualenvs/ice_breaker-p8pkssDf/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1685: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [6]:
# Query Router and Routing Logic

class RouteQuery(BaseModel):
    datasource: Literal["vectorstore", "websearch"] = Field(...)

question_router = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at routing a user question to a vectorstore or web search."),
    ("human", "{question}"),
]) | llm.with_structured_output(RouteQuery)

def route_question(state: GraphState) -> str:
    print("---ROUTE QUESTION---")
    question = state["question"]
    source: RouteQuery = question_router.invoke({"question": question})
    if source.datasource == "websearch":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    print("---ROUTE QUESTION TO RAG---")
    return "retrieve"

def decide_to_generate(state: GraphState) -> str:
    print("---ASSESS GRADED DOCUMENTS---")
    return "websearch" if state["web_search"] else "generate"

def grade_generation_grounded_in_documents_and_question(state: GraphState) -> str:
    print("---CHECK HALLUCINATIONS---")
    if hallucination_grader.invoke({"documents": state["documents"], "generation": state["generation"]}).binary_score:
        print("---GENERATION IS GROUNDED---")
        if answer_grader.invoke({"question": state["question"], "generation": state["generation"]}).binary_score:
            print("---GENERATION IS USEFUL---")
            return "useful"
        print("---GENERATION NOT USEFUL---")
        return "not useful"
    print("---GENERATION NOT GROUNDED---")
    return "not supported"

In [15]:
# Build LangGraph Workflow

RETRIEVE = "retrieve"
GRADE_DOCUMENTS = "grade_documents"
GENERATE = "generate"
WEBSEARCH = "websearch"

workflow = StateGraph(GraphState)
workflow.add_node(RETRIEVE, retrieve)
workflow.add_node(GRADE_DOCUMENTS, grade_documents)
workflow.add_node(GENERATE, generate)
workflow.add_node(WEBSEARCH, web_search)
workflow.set_conditional_entry_point(route_question, {
    WEBSEARCH: WEBSEARCH,
    RETRIEVE: RETRIEVE,
})
workflow.add_edge(RETRIEVE, GRADE_DOCUMENTS)
workflow.add_conditional_edges(GRADE_DOCUMENTS, decide_to_generate, {
    WEBSEARCH: WEBSEARCH,
    GENERATE: GENERATE,
})
workflow.add_conditional_edges(GENERATE, grade_generation_grounded_in_documents_and_question, {
    "not supported": GENERATE,
    "useful": END,
    "not useful": WEBSEARCH,
})
workflow.add_edge(WEBSEARCH, GENERATE)
workflow.add_edge(GENERATE, END)
app = workflow.compile()

#output graph structure
from IPython.display import Image
print("LangGraph Workflow Structure:")

app.get_graph().draw_mermaid_png(output_file_path="graph.png")
Image(filename="graph.png")
print(app.get_graph().draw_mermaid())

LangGraph Workflow Structure:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve(retrieve)
	grade_documents(grade_documents)
	generate(generate)
	websearch(websearch)
	__end__([<p>__end__</p>]):::last
	__start__ -.-> retrieve;
	__start__ -.-> websearch;
	generate -. &nbsp;useful&nbsp; .-> __end__;
	generate -. &nbsp;not useful&nbsp; .-> websearch;
	grade_documents -.-> generate;
	grade_documents -.-> websearch;
	retrieve --> grade_documents;
	websearch --> generate;
	generate -. &nbsp;not supported&nbsp; .-> generate;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [16]:
# Run App with User Input

user_question = input("Enter your question: ")
result = app.invoke({
    "question": user_question,
    "documents": [],
    "generation": "",
    "web_search": False
})
print("\n\nFinal Answer:", result.get("generation"))

Enter your question:  What is agent memory?


---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---GENERATION IS GROUNDED---
---GENERATION IS USEFUL---


Final Answer: Agent memory refers to the ability of AI agents to retain information over time, allowing them to behave more intelligently by remembering prior conversations. It enables context-aware AI behavior and stores persistent knowledge that can be used across sessions or tasks. Agent memory empowers AI systems to store and recall past experiences to improve decision-making and overall performance.
